In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
# Load data
data_path = '/AFAD_1900_today_31461754_4_9.csv'
df = pd.read_csv(data_path)
# Changed the format string to match the date format in your data
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M:%S')

# Extract datetime components
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['Previous_Magnitude'] = df['Magnitude'].shift(1).fillna(method='bfill')  # Fill the first missing value

# Select relevant features for the model
features = ['Latitude', 'Longitude', 'Magnitude', 'Year', 'Month', 'Day', 'Hour', 'Previous_Magnitude']

In [ ]:
df.head()

,Date,Longitude,Latitude,Depth,Type,Magnitude,Location,EventID,TimeName,TypeName,MagnitudeName,Year,Month,Day,Hour,Previous_Magnitude
0,1903-04-29 16:07:56,42.65,39.15,33.0,MS,5.0,-,7821,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,4,29,16,5.0
1,1903-04-30 04:12:56,42.65,39.15,10.0,MS,4.6,-,7822,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,4,30,4,5.0
2,1903-05-03 01:56:56,41.50,38.70,10.0,MS,4.9,-,7823,Ayhan ve Diğ.1981,Ayhan ve Diğ.1981,Ayhan ve Diğ.1981,1903,5,3,1,4.6
3,1903-05-21 20:56:56,42.50,39.00,10.0,MS,4.2,-,7824,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,5,21,20,4.9
4,1903-05-26 08:05:56,29.00,40.65,10.0,MS,5.9,-,7825,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,Ambraseys-Finkel 1987,1903,5,26,8,4.2


In [ ]:
def create_sequences(df, features, window_size):
    sequences = []
    labels = []
    for i in range(len(df) - window_size):
        sequence = df.iloc[i:i+window_size][features].values
        label = df.iloc[i+window_size][['Latitude', 'Longitude', 'Magnitude']].values
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

window_size = 100  # Number of past days to consider
X, y = create_sequences(df, features, window_size)

In [ ]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

# Convert y to numerical values before creating a tensor
y_numeric = y.astype(np.float32)  # Assuming all elements in y should be numerical
y_tensor = torch.tensor(y_numeric, dtype=torch.float32)

# DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        return self.fc(out)

# Initialize the model
model = LSTMModel(input_dim=len(features), hidden_dim=50, num_layers=1, output_dim=3)

In [ ]:
# @title  { display-mode: "code" }
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for seq, labels in dataloader:
        model.train()
        optimizer.zero_grad()
        outputs = model(seq)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 1 == 0:  # Print loss every 10 epochs
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 24.05324363708496
Epoch 2/50, Loss: 22.386808395385742
Epoch 3/50, Loss: 17.315570831298828
Epoch 4/50, Loss: 20.94799041748047
Epoch 5/50, Loss: 9.511486053466797
Epoch 6/50, Loss: 17.195331573486328
Epoch 7/50, Loss: 26.261539459228516
Epoch 8/50, Loss: 26.905611038208008
Epoch 9/50, Loss: 19.216676712036133
Epoch 10/50, Loss: 19.164827346801758
Epoch 11/50, Loss: 15.499706268310547
Epoch 12/50, Loss: 12.959505081176758
Epoch 13/50, Loss: 9.023143768310547
Epoch 14/50, Loss: 24.36623191833496
Epoch 15/50, Loss: 14.075852394104004
Epoch 16/50, Loss: 12.911050796508789
Epoch 17/50, Loss: 26.153934478759766
Epoch 18/50, Loss: 20.563329696655273
Epoch 19/50, Loss: 15.325235366821289
Epoch 20/50, Loss: 13.726975440979004
Epoch 21/50, Loss: 20.271211624145508
Epoch 22/50, Loss: 20.632658004760742
Epoch 23/50, Loss: 19.9948787689209
Epoch 24/50, Loss: 21.636314392089844
Epoch 25/50, Loss: 20.648929595947266
Epoch 26/50, Loss: 11.552887916564941
Epoch 27/50, Loss: 23.379705

In [ ]:
model.eval()
with torch.no_grad():
    for seq, labels in dataloader:
        predictions = model(seq)
        # Compare predictions to actual labels
        # print("Predictions:", predictions)
        # print("Actual:", labels)

In [ ]:
model.eval()  # Set the model to evaluation mode
input_seq = X_tensor[-1].unsqueeze(0)  # Start with the last sequence from the dataset

predictions = []
num_predictions = 10000

with torch.no_grad():
    for _ in range(num_predictions):
        # Predict the next earthquake
        output = model(input_seq)
        predictions.append(output.numpy().flatten())  # Store the prediction

        # Prepare the next input sequence
        # Assume 'output' provides latitude, longitude, and magnitude
        # and the rest of the features are carried over or slightly modified from the last input
        new_features = np.zeros(input_seq.shape[-1])  # Start with a zeroed array for the new features
        new_features[:3] = output.numpy().flatten()  # Set the first three features from the model output
        new_features[3:] = input_seq.numpy().squeeze(0)[-1, 3:]  # Carry forward the rest (or modify date components if necessary)

        # Update 'Previous Magnitude' to the last predicted magnitude
        new_features[-1] = output.numpy().flatten()[2]  # Assuming 'Magnitude' is the third output

        # Append to the input sequence and remove the oldest entry
        new_seq = torch.cat((input_seq.squeeze(0)[1:], torch.tensor([new_features], dtype=torch.float32)), dim=0)
        input_seq = new_seq.unsqueeze(0)

# Optionally, reverse any scaling on predictions if necessary
predicted_data = np.array(predictions)
predicted_lats = predicted_data[:, 0]  # Assuming Latitude is the first output
predicted_lons = predicted_data[:, 1]  # Assuming Longitude is the second output
predicted_mags = predicted_data[:, 2]  # Assuming Magnitude is the third output

In [ ]:
import numpy as np

# Run the model and collect predictions
model.eval()  # Set the model to evaluation mode
predictions = []
actuals = []

with torch.no_grad():
    for seq, labels in dataloader:
        output = model(seq)
        predictions.append(output.numpy())
        actuals.append(labels.numpy())

# Convert lists to arrays for easier handling
predictions = np.vstack(predictions)
actuals = np.vstack(actuals)

# Select a random subset of 10 pairs
num_samples = 10
indices = np.random.choice(predictions.shape[0], num_samples, replace=False)
selected_predictions = predictions[indices]
selected_actuals = actuals[indices]


In [ ]:
# Select random samples from both datasets
num_samples = 10  # number of samples to display on the map
indices = np.random.choice(predicted_data.shape[0], num_samples, replace=False)
predicted_samples = predicted_data[indices, :]
actual_samples = actual_data[indices, :]

In [ ]:
import folium

# Initialize the map centered around Turkey
map = folium.Map(location=[39, 35.5], zoom_start=6)

# Plot each pair of predicted and actual data
for pred, actual in zip(selected_predictions, selected_actuals):
    predicted_lat, predicted_lon, predicted_mag = pred
    actual_lat, actual_lon, actual_mag = actual

    # Marker for predicted earthquake
    folium.Marker(
        location=[predicted_lat, predicted_lon],
        icon=folium.Icon(color='blue', icon='cloud'),
        popup=f"Predicted Magnitude: {predicted_mag:.2f}"
    ).add_to(map)

    # Marker for actual earthquake
    folium.Marker(
        location=[actual_lat, actual_lon],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup=f"Actual Magnitude: {actual_mag:.2f}"
    ).add_to(map)

    # Optionally draw a line between the predicted and actual points
    folium.PolyLine(locations=[[predicted_lat, predicted_lon], [actual_lat, actual_lon]], color='gray').add_to(map)

# Save or display the map
map.save('selected_earthquake_predictions.html')
